In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from collections import defaultdict

In [3]:
usecols = ['ncodpers', 'ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'ind_cco_fin_ult1',
       'ind_cder_fin_ult1', 'ind_cno_fin_ult1', 'ind_ctju_fin_ult1',
       'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1',
       'ind_deco_fin_ult1', 'ind_deme_fin_ult1', 'ind_dela_fin_ult1',
       'ind_ecue_fin_ult1', 'ind_fond_fin_ult1', 'ind_hip_fin_ult1',
       'ind_plan_fin_ult1', 'ind_pres_fin_ult1', 'ind_reca_fin_ult1',
       'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1', 'ind_viv_fin_ult1',
       'ind_nomina_ult1', 'ind_nom_pens_ult1', 'ind_recibo_ult1']

In [19]:
#df_train = pd.read_csv('train_ver2.csv', usecols=usecols, nrows=7000000)
sample = pd.read_csv('sample_submission.csv', nrows = 100)

In [6]:
df_train = df_train.drop_duplicates(['ncodpers'], keep='last')

In [7]:
df_train.fillna(0, inplace=True)

In [8]:
models = {}
model_preds = {}

In [9]:
id_preds = defaultdict(list)

In [10]:
ids = df_train['ncodpers'].values

In [13]:
for c in df_train.columns:
    if c!= 'ncodpers' :
        print(c)
        y_train = df_train[c]
        x_train = df_train.drop([c, 'ncodpers'], 1)
        clf = LogisticRegression()
        clf.fit(x_train, y_train)
        p_train = clf.predict_proba(x_train)[:,1]
        
        models[c] = clf
        model_preds[c] = p_train
        for id, p in zip(ids, p_train):
            id_preds[id].append(p)
        
        print(roc_auc_score(y_train, p_train))   
        

ind_ahor_fin_ult1
0.80899342
ind_aval_fin_ult1
0.982231304931
ind_cco_fin_ult1
0.645184638818
ind_cder_fin_ult1
0.780425218234
ind_cno_fin_ult1
0.975975743989
ind_ctju_fin_ult1
0.871581911298
ind_ctma_fin_ult1
0.691373709082
ind_ctop_fin_ult1
0.609121295155
ind_ctpp_fin_ult1
0.724824356456
ind_deco_fin_ult1
0.621992963603
ind_deme_fin_ult1
0.856226187346
ind_dela_fin_ult1
0.778887635629
ind_ecue_fin_ult1
0.785597188521
ind_fond_fin_ult1
0.857678914772
ind_hip_fin_ult1
0.922229634454
ind_plan_fin_ult1
0.8464687116
ind_pres_fin_ult1
0.713859409195
ind_reca_fin_ult1
0.845247045346
ind_tjcr_fin_ult1
0.905090361334
ind_valo_fin_ult1
0.83668013566
ind_viv_fin_ult1
0.764697879545
ind_nomina_ult1
0.998183076205
ind_nom_pens_ult1
0.998820052244
ind_recibo_ult1
0.872553280971


In [14]:
already_active = {}
for row in df_train.values:
    row = list(row)
    id = row.pop(0)
    active = [c[0] for c in zip(df_train.columns[1:], row) if c[1] > 0]
    already_active[id] = active

In [15]:
train_preds = {}
for id, p in id_preds.items():
    # Here be dragons
    preds = [i[0] for i in sorted([i for i in zip(df_train.columns[1:], p) if i[0] not in already_active[id]], key=lambda i:i [1], reverse=True)[:7]]
    train_preds[id] = preds

In [20]:
test_preds = []
for row in sample.values:
    id = row[0]
    p = train_preds[id]
    test_preds.append(' '.join(p))

In [21]:
sample['added_products'] = test_preds
sample.to_csv('collab_sub.csv', index=False)

In [ ]:
#https://www.kaggle.com/anokas/collaborative-filtering-btb-lb-0-01691